# Example usage of segmentation to btrack to napari visualization

This example uses TIF files saved out from segmentation using *stardist3D*, although will work for other segmentation pipelines too.

In [1]:
import glob
import os

import btrack
import napari

import numpy as np

from skimage.io import imread

In [2]:
PATH = "/media/quantumjot/Data/Test"

In [3]:
files = glob.glob(os.path.join(PATH, '*.tif'))

# sort the files numerically
files = sorted(files, key=lambda f: int(f[len(os.path.join(PATH, 'labels_')):-4]))

In [4]:
files

['/media/quantumjot/Data/Test/labels_0.tif',
 '/media/quantumjot/Data/Test/labels_1.tif',
 '/media/quantumjot/Data/Test/labels_2.tif',
 '/media/quantumjot/Data/Test/labels_3.tif',
 '/media/quantumjot/Data/Test/labels_4.tif',
 '/media/quantumjot/Data/Test/labels_5.tif',
 '/media/quantumjot/Data/Test/labels_6.tif',
 '/media/quantumjot/Data/Test/labels_7.tif',
 '/media/quantumjot/Data/Test/labels_8.tif',
 '/media/quantumjot/Data/Test/labels_9.tif']

### method 1 - using a numpy array

In this example, each image from the timelapse is a 3D volume (32 x 1200 x 1200) and there are 10 timepoints

In [5]:
def segmentation_arr(files):
    """Segmentation as numpy array."""
    
    stack = []
    for filename in files:
        img = imread(filename)
        stack.append(img)
    return np.stack(stack, axis=0)

In [6]:
stack = segmentation_arr(files)

Now we print out the shape of the stack (T, Z, Y, X):

In [7]:
stack.shape

(10, 32, 1200, 1200)

### method 2 - using a generator

This is useful if you're resource constrained and don't want to load all of the image data, or they are stored in an unusual format. Note that the generator produces a numpy array for each image...

In [8]:
def segmentation_generator(files):
    """Segmentation generator"""
    
    for filename in files:
        img = imread(filename)
        yield img

In [9]:
generator = segmentation_generator(files)

## localizing the objects

Now we use a utility function to localise the objects in the segmentation, and also apply anisotropic scaling (using the `scale` option, here the z-values are scaled by 2x). Note that we can also use scikit-image `regionprops` to calculate properties for each object, using the `properties` keyword:

In [10]:
obj_from_arr = btrack.utils.segmentation_to_objects(stack, properties=('area', ), scale=(2., 1., 1.))

[INFO][2021/04/19 01:39:56 PM] Localizing objects from segmentation...
[INFO][2021/04/19 01:40:02 PM] Objects are of type: <class 'dict'>
[INFO][2021/04/19 01:40:02 PM] ...Found 11625 objects in 10 frames.


In [11]:
# inspect the first object
obj_from_arr[0]

,ID,x,y,z,t,dummy,states,label,prob,area
0,0,939.0,20.0,12.0,0,False,0,5,0.0,1381


In [12]:
obj_from_generator = btrack.utils.segmentation_to_objects(
    generator, 
    properties = ('area', 'major_axis_length'), 
    scale=(2., 1., 1.)
)

[INFO][2021/04/19 01:40:02 PM] Localizing objects from segmentation...
[INFO][2021/04/19 01:40:12 PM] Objects are of type: <class 'dict'>
[INFO][2021/04/19 01:40:12 PM] ...Found 11625 objects in 10 frames.


In [13]:
# inspect the first object
obj_from_generator[0]

,ID,x,y,z,t,dummy,states,label,prob,area,major_axis_length
0,0,939.0,20.0,12.0,0,False,0,5,0.0,1381,26.792221


## run btrack with the objects

We will use the objects from the generator here.

In [14]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file('../models/cell_config.json')
    tracker.max_search_radius = 10

    # append the objects to be tracked
    tracker.append(obj_from_generator)

    # set the volume
    tracker.volume=((0, 1200), (0, 1200), (-1e5, 64.))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=100)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

    tracker.export(os.path.join(PATH, 'tracking.h5'), obj_type='obj_type_1')

    # get the tracks in a format for napari visualization
    data, properties, graph = tracker.to_napari(ndim=3)
    
    tracks = tracker.tracks

[INFO][2021/04/19 01:40:13 PM] Loaded btrack: /home/quantumjot/Dropbox/Code/py3/BayesianTracker/btrack/libs/libtracker.so
[INFO][2021/04/19 01:40:13 PM] btrack (v0.4.1) library imported
[INFO][2021/04/19 01:40:13 PM] Setting max XYZ search radius to: 100
[INFO][2021/04/19 01:40:13 PM] Starting BayesianTracker session
[INFO][2021/04/19 01:40:13 PM] Loading configuration file: ../models/cell_config.json
[INFO][2021/04/19 01:40:13 PM] Loading motion model: b'cell_motion'
[INFO][2021/04/19 01:40:13 PM] Setting max XYZ search radius to: 10
[INFO][2021/04/19 01:40:13 PM] Objects are of type: <class 'list'>
[INFO][2021/04/19 01:40:13 PM] Set volume to ((0, 1200), (0, 1200), (-100000.0, 64.0))
[INFO][2021/04/19 01:40:13 PM] Starting tracking... 
[INFO][2021/04/19 01:40:13 PM] Tracking objects in frames 0 to 10 (of 10)...
[INFO][2021/04/19 01:40:21 PM]  - Timing (Bayesian updates: 923.01ms, Linking: 9.44ms)
[INFO][2021/04/19 01:40:21 PM]  - Probabilities (Link: 1.00000, Lost: 0.52636)
[INFO][20

## visualize with napari

Note that we also set the scale of the images here to account for the anisotropy.

In [15]:
viewer = napari.Viewer()
viewer.add_labels(stack, scale=(1., 2., 1., 1.), name='Segmentation')
viewer.add_tracks(data, properties=properties, graph=graph, name='Tracks')

<Tracks layer 'Tracks' at 0x7fdf504b1150>